In [188]:
import os
import time
import math
from datetime import datetime
import blpapi
import flatbuffers
from BlpConn.Message import HeadlineCalendarEvent
from BlpConn.Message import HeadlineEconomicEvent
from BlpConn.Message.EventType import EventType
from dotenv import load_dotenv
load_dotenv()

True

In [14]:
service = "//blp/economic-data"

In [6]:
client_cert_path = os.environ["CLIENT_CERTIFICATE"]
client_cert_password = os.environ["PASSWORD"]
root_cert_path = os.environ["ROOT_CERTIFICATE"]

In [8]:
tls_options = blpapi.TlsOptions.createFromFiles(
    client_cert_path,
    client_cert_password,
    root_cert_path
)

In [10]:
host_primary = os.environ["PRIMARY_HOST"]
host_secondary = os.environ["SECONDARY_HOST"]
port = int(os.environ["PORT"])

In [15]:
session_options = blpapi.SessionOptions()
session_options.setServerAddress(host_primary, port, 0)
session_options.setServerAddress(host_secondary, port, 0)
session_options.setTlsOptions(tls_options)
session_options.setAutoRestartOnDisconnection(True)
session_options.setNumStartAttempts(3)
session_options.setDefaultSubscriptionService(service)

In [21]:
session = blpapi.Session(session_options)
session.start()

True

In [22]:
session.openService(service)

True

In [24]:
sub = blpapi.SubscriptionList()
sub.add("//blp/economic-data/headline-actuals/ticker/CATBTOTB Index")
corr_id = session.subscribe(sub)

In [25]:

event = session.nextEvent()
event

In [26]:
event.eventType()

2

In [37]:
event = session.nextEvent()
event.eventType()

8

In [105]:
messages = [msg for msg in event]
elem = messages[0].asElement()

In [204]:
def getSringValue(elem: blpapi.Element, name: str):
    if elem.hasElement(name):
        return elem.getElement(name).getValue(0)
    else:
        return ""


def getDateTimeValue(elem: blpapi.Element, name: str):
    if not elem.hasElement(name):
        return None
    return elem.getElement(name).getValue(0)

def convertToMicroOffset(dt: datetime):
    return {
        "micros": dt.timestamp(),
        "offset": 0
    }
    

def getReleaseTime(elem: blpapi.Element, name: str):
    if not elem.hasElement(name):
        return None
    sub_elem = elem.getElement(name)
    if sub_elem.hasElement("DATETIME"):
        dt = getDateTimeValue(sub_elem, "DATETIME")
        return {
            "start": dt,
            "end": dt
        }
    elif sub_elem.hasElement("DATE"):
        dt = getDateTimeValue(sub_elem, "DATETIME")
        return {
            "start": dt,
            "end": dt
        }
    elif sub_elem.hasElement("DATERANGE"):
        data_range = sub_elem.getElement("DATERANGE")
        return {
            "start": getDateTimeValue(date_range, "START"),
            "end": getDateTimeValue(date_range, "END"),
        }
    return None

def getFloatValue(elem: blpapi.Element, name: str):
    if not elem.hasElement(name):
        return math.nan
    return float(elem.getElement(name).getValue(0))

def getSingleValue(elem: blpapi.Element):
    return {
        "single": getFloatValue(elem, "SINGLE"),
    }

def getDistributionValue(elem: blpapi.Element, name: str):
    return {
        "low": getFloatValue(elem, "LOW"),
        "high": getFloatValue(elem, "HIGH"),
        "median": getFloatValue(elem, "MEDIAN"),
        "average": getFloatValue(elem, "AVERAGE"),
        "standard_deviation": getFloatValue(elem, "STANDARD_DEVIATION"),
        "number": getFloatValue(elem, "NUMBER"),
    }

def getValue(elem: blpapi.Element, name: str):
    if not elem.hasElement(name):
        return None
    sub_elem = elem.getElement(name)
    print(">>>> ", sub_elem)
    if sub_elem.hasElement("SINGLE"):
        return getFloatValue(sub_elem, "SINGLE")
    elif sub_elem.hasElement("DISTRIBUTION"):
        return getDistributionValue(sub_elem.getElement("DISTRIBUTION"))
    return None

def getRevisionMetadata(elem: blpapi.Element, name: str):
    if not elem.hasElement(name):
        return None
    sub_elem = elem.getElement(name)
    return {
        "prior_event_id": sub_elem.getElement("PRIOR_EVENT_ID").getValue(0),
        "prior_observation_period": getSringValue(sub_elem, "PRIOR_OBSERVATION_PERIOD"),
        "prior_economic_release": getReleaseTime(sub_elem, "PRIOR_ECONOMIC_RELEASE"),
    }

def parseHeadlineEconomicEvent(elem: blpapi.Element):
    builder = flatbuffers.Builder()
    HeadlineCalendarEvent.Start(builder)
    HeadlineEconomicEvent.AddIdBbGlobal(builder.CreateString(getSringValue(elem, "ID_BB_GLOBAL")))
    HeadlineEconomicEvent.AddParsekyableDes(builder.CreateString(getSringValue(elem, "PARSEKYABLE_DES")))
    HeadlineEconomicEvent.AddDescription(builder.CreateString(getSringValue(elem, "DESCRIPTION")))
    HeadlineEconomicEvent.AddEventType(builder.CreateString(getSringValue(elem, "EVENTYPE")))
    HeadlineEconomicEvent.AddEventSubtype(builder.CreateString(getSringValue(elem, "EVENT_SUBTYPE")))
    HeadlineEconomicEvent.AddEventId(elem.getElement("EVENT_ID").getValue(0))
    HeadlineEconomicEvent.AddObservationPeriod(builder.CreateString(getSringValue(elem, "OBSERVATION_PERIOD")))
    HeadlineEconomicEvent.End(builder)

def parseHeadlineCalendarEvent(elem: blpapi.Element):
    return {
        "id_bb_global": getSringValue(elem, "ID_BB_GLOBAL"),
        "parsekyable_des": getSringValue(elem, "PARSEKYABLE_DES"),
        "description": getSringValue(elem, "DESCRIPTION"),
        "event_type": getSringValue(elem, "EVENTYPE"),
        "event_subtype": getSringValue(elem, "EVENT_SUBTYPE"),
        "event_id": elem.getElement("EVENT_ID").getValue(0),
        "observation_period": getSringValue(elem, "OBSERVATION_PERIOD"),
        "eco_release_dt": getReleaseTime(elem, "ECO_RELEASE_DT"),
        "release_status": getSringValue(elem, "RELEASE_STATUS"),
    } 

def parse_economic_event(elem: blpapi.Element):
    if elem.hasElement("HeadlineEconomicEvent"):
        return parseHeadlineEconomicEvent(elem.getElement("HeadlineEconomicEvent"))
    elif elem.hasElement("HeadlineCalendarEvent"):
        return parseHeadlineCalendarEvent(elem.getElement("HeadlineCalendarEvent"))
    return None

In [205]:
print(parse_economic_event(elem))
print(elem)

IsNestedError: 

In [98]:
et == blpapi.Name('ACTUAL')

True

In [ ]:
HeadlineCalendarEvent.HeadlineCalendarEventStart

In [174]:
ee = EconomicEvent()
ee.